#### Modify the config

In [26]:


from mmcv import Config
root_path = '/home/zhsong/Code/'
cfg = Config.fromfile(root_path+'mmdetection/configs/yolox/yolox_s_8x8_300e_coco.py')



#### Data visdrone2019

In [27]:
from mmdet.apis import set_random_seed

# Modify dataset type and path
root_path = '/home/zhsong/Code/'
classes = ('pedestrian', 'people','bicycle', 'car', 'van','truck','tricycle','awning-tricycle','bus', 'motor')

cfg.dataset_type = 'CocoDataset'
cfg.data_root = root_path + 'datasets/VisDrone2019-DET/'

cfg.data.test.type = 'CocoDataset'
cfg.data.test.data_root = cfg.data_root
cfg.data.test.ann_file = cfg.data_root + 'annotations/VisDrone2019-DET-test-dev-coco.json'
cfg.data.test.img_prefix = cfg.data_root + 'VisDrone2019-DET-test-dev/images/'
cfg.data.test.classes = classes

# cfg.train_dataset.dataset.data_root = cfg.data_root
cfg.train_dataset.dataset.ann_file = cfg.data_root + 'annotations/VisDrone2019-DET-train-coco.json'
cfg.train_dataset.dataset.img_prefix =  cfg.data_root + 'VisDrone2019-DET-train/images/'
cfg.train_dataset.dataset.classes = classes
cfg.data.train=cfg.train_dataset

cfg.data.val.type = 'CocoDataset'
cfg.data.val.data_root = cfg.data_root
cfg.data.val.ann_file = cfg.data_root + 'annotations/VisDrone2019-DET-val-coco.json'
cfg.data.val.img_prefix =  cfg.data_root + 'VisDrone2019-DET-val/images/'
cfg.data.val.classes = classes
# modify num classes of the model in box head
cfg.model.bbox_head.num_classes = 10
# We can still use the pre-trained Mask RCNN model though we do not need to
# use the mask branch
cfg.load_from = root_path + 'mmdetection/checkpoints/yolox_s_8x8_300e_coco_20211121_095711-4592a793.pth'

# Set up working dir to save files and logs.
cfg.work_dir = root_path + 'mmdetection/exps_out/yolox_visdrone'

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optimizer.lr = 0.02 / 8
cfg.lr_config.warmup = None
cfg.log_config.interval = 10

# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = 'mAP'
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 1
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 1

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.max_epochs = 10
cfg.runner.max_epochs = cfg.max_epochs
cfg.evaluation.metric = 'bbox'
cfg.evaluation.classwise = True
# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

Config:
optimizer = dict(
    type='SGD',
    lr=0.0025,
    momentum=0.9,
    weight_decay=0.0005,
    nesterov=True,
    paramwise_cfg=dict(norm_decay_mult=0.0, bias_decay_mult=0.0))
optimizer_config = dict(grad_clip=None)
lr_config = dict(
    policy='YOLOX',
    warmup=None,
    by_epoch=False,
    warmup_by_epoch=True,
    warmup_ratio=1,
    warmup_iters=5,
    num_last_epochs=15,
    min_lr_ratio=0.05)
runner = dict(type='EpochBasedRunner', max_epochs=10)
checkpoint_config = dict(interval=1)
log_config = dict(interval=10, hooks=[dict(type='TextLoggerHook')])
custom_hooks = [
    dict(type='YOLOXModeSwitchHook', num_last_epochs=15, priority=48),
    dict(type='SyncNormHook', num_last_epochs=15, interval=10, priority=48),
    dict(
        type='ExpMomentumEMAHook',
        resume_from=None,
        momentum=0.0001,
        priority=49)
]
dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = '/home/zhsong/Code/mmdetection/checkpoints/yolox_s_8x8_300e_coco_20211121_0957

#### Train yolox

Finally, lets initialize the dataset and detector, then train a new detector!

In [28]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
import mmcv
import os.path as osp

# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

loading annotations into memory...
Done (t=2.95s)
creating index...
index created!


2022-03-09 21:59:16,809 - mmdet - INFO - load checkpoint from local path: /home/zhsong/Code/mmdetection/checkpoints/yolox_s_8x8_300e_coco_20211121_095711-4592a793.pth
2022-03-09 21:59:16,851 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for bbox_head.multi_level_conv_cls.0.weight: copying a param with shape torch.Size([80, 128, 1, 1]) from checkpoint, the shape in current model is torch.Size([10, 128, 1, 1]).
size mismatch for bbox_head.multi_level_conv_cls.0.bias: copying a param with shape torch.Size([80]) from checkpoint, the shape in current model is torch.Size([10]).
size mismatch for bbox_head.multi_level_conv_cls.1.weight: copying a param with shape torch.Size([80, 128, 1, 1]) from checkpoint, the shape in current model is torch.Size([10, 128, 1, 1]).
size mismatch for bbox_head.multi_level_conv_cls.1.bias: copying a param with shape torch.Size([80]) from checkpoint, the shape in current model is torch.Size([10]).
size mismatch for bbox_

loading annotations into memory...
Done (t=0.09s)
creating index...
index created!


2022-03-09 21:59:24,624 - mmdet - INFO - Epoch [1][10/809]	lr: 1.250e-04, eta: 1:44:24, time: 0.775, data_time: 0.372, memory: 6092, loss_cls: 4.7215, loss_bbox: 3.0114, loss_obj: 4.2115, loss: 11.9445
2022-03-09 21:59:28,889 - mmdet - INFO - Epoch [1][20/809]	lr: 1.250e-04, eta: 1:20:49, time: 0.427, data_time: 0.016, memory: 6092, loss_cls: 2.1917, loss_bbox: 2.8742, loss_obj: 2.6011, loss: 7.6670
2022-03-09 21:59:33,007 - mmdet - INFO - Epoch [1][30/809]	lr: 1.250e-04, eta: 1:12:14, time: 0.412, data_time: 0.013, memory: 6092, loss_cls: 1.8560, loss_bbox: 2.8108, loss_obj: 2.4508, loss: 7.1176
2022-03-09 21:59:36,086 - mmdet - INFO - Epoch [1][40/809]	lr: 1.250e-04, eta: 1:04:25, time: 0.307, data_time: 0.017, memory: 6092, loss_cls: 1.7836, loss_bbox: 3.1434, loss_obj: 2.4873, loss: 7.4143
2022-03-09 21:59:39,902 - mmdet - INFO - Epoch [1][50/809]	lr: 1.250e-04, eta: 1:01:43, time: 0.382, data_time: 0.020, memory: 6092, loss_cls: 1.6126, loss_bbox: 3.0049, loss_obj: 2.3606, loss: 6

[                               ] 12/548, 31.9 task/s, elapsed: 0s, ETA:    17s

...

2022-03-09 23:02:49,147 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=1.07s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=69.49s).
Accumulating evaluation results...


2022-03-09 23:04:02,581 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.095
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.175
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.091
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.040
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.158
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.356
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.159
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.162
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.162
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.071
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.272
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000

DONE (t=2.37s).


2022-03-09 23:04:02,892 - mmdet - INFO - Now best checkpoint is saved as best_bbox_mAP_epoch_10.pth.
2022-03-09 23:04:02,893 - mmdet - INFO - Best bbox_mAP is 0.0950 at 10 epoch.
2022-03-09 23:04:02,894 - mmdet - INFO - Epoch(val) [10][548]	bbox_mAP: 0.0950, bbox_mAP_50: 0.1750, bbox_mAP_75: 0.0910, bbox_mAP_s: 0.0400, bbox_mAP_m: 0.1580, bbox_mAP_l: 0.3560, bbox_mAP_copypaste: 0.095 0.175 0.091 0.040 0.158 0.356
